# Quantitative Value Strategy
## Library Imports

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import requests
from scipy import stats
import math

## Importing List of Stocks & API Token

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'EASQGTLCS SA) E(NE/LMB DKAGAONLRT',
 'calculationPrice': 'close',
 'open': None,
 'openTime': None,
 'openSource': 'icaioffl',
 'close': None,
 'closeTime': None,
 'closeSource': 'ilfoafci',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 126.78,
 'latestSource': 'Close',
 'latestTime': 'March 9, 2021',
 'latestUpdate': 1630861138144,
 'latestVolume': None,
 'iexRealtimePrice': 0,
 'iexRealtimeSize': 0,
 'iexLastUpdated': 0,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 119.3,
 'previousVolume': 156772209,
 'change': 4.757,
 'changePercent': 0.04175,
 'volume': None,
 'iexMarketPercent': None,
 'iexVolume': 0,
 'avgTotalVolume': 115771022,
 'iexBidPrice':

## Parsing API Call

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']


## Executing Batch API Call & Building DataFrame

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

##Create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,122.560,47.62,N/A
1,AAL,21.900,-1.24,N/A
2,AAP,180.970,25.32,N/A
3,AAPL,124.073,34,N/A
4,ABBV,109.890,41.2,N/A
...,...,...,...,...
500,YUM,104.390,35.57,N/A
501,ZBH,162.820,-241.31,N/A
502,ZBRA,478.330,52.59,N/A
503,ZION,59.270,8.98,N/A


## Removing Glamour Stocks

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0][:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace = True)


## Calculating the Number of Shares to Buy

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

`portfolio_input` function to accept a `portfolio_size` 

In [9]:
portfolio_input()

Enter the value of your portfolio:25000000


In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe    

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,569.24,4.54,878
1,AIV,5.02,4.79,99601
2,ALL,112.19,6.48,4456
3,EBAY,56.40,7.29,8865
4,AFL,51.60,7.6,9689
5,UNM,29.60,7.63,16891
6,CPB,48.39,8.35,10332
7,KIM,19.84,8.8,25201
8,COO,384.00,8.84,1302
9,ZION,59.27,8.98,8435


## Building a Better (and More Realistic) Value Strategy

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
#print(data['AAPL']['advanced-stats'])
#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
#Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']
#Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']
#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gp = enterprise_value/gross_profit


###`rv` = `robust value`

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gp = enterprise_value/gross_profit
        except:
            ev_to_gp = np.NaN    
        
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gp,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
    
rv_dataframe    


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.08,N/A,48.07,N/A,7.6,N/A,6.7,N/A,31.154625,N/A,13.060984,N/A,N/A
1,AAL,21.60,N/A,-1.19,N/A,-2.09,N/A,0.82,N/A,-4.367005,N/A,2.312093,N/A,N/A
2,AAP,176.88,N/A,24.64,N/A,3.2,N/A,1.15,N/A,11.649829,N/A,2.640324,N/A,N/A
3,AAPL,126.79,N/A,34.2,N/A,32.2,N/A,7.24,N/A,24.378196,N/A,18.617930,N/A,N/A
4,ABBV,109.68,N/A,39.32,N/A,14.73,N/A,4.31,N/A,14.145450,N/A,8.784794,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.91,N/A,35.14,N/A,-4.1,N/A,5.56,N/A,23.927399,N/A,10.198238,N/A,N/A
501,ZBH,162.39,N/A,-240.98,N/A,2.67,N/A,4.64,N/A,29.243242,N/A,8.119188,N/A,N/A
502,ZBRA,473.53,N/A,51.5,N/A,11.81,N/A,5.73,N/A,33.214935,N/A,13.167546,N/A,N/A
503,ZION,57.51,N/A,9.25,N/A,1.33,N/A,2.68,N/A,7.056707,N/A,3.018850,N/A,N/A


## Dealing With Missing Data in DataFrame

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,240.50,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
42,APA,22.50,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,263.98,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.87,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,55.49,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.47,N/A,14.37,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,57.17,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.59,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,172.60,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2130.60,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

In [16]:
from scipy.stats import percentileofscore as score
metrics = { 
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
        
rv_dataframe        

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.08,N/A,48.07,0.829703,7.60,0.737624,6.70,0.775248,31.154625,0.879208,13.060984,0.782178,N/A
1,AAL,21.60,N/A,-1.19,0.156436,-2.09,0.0455446,0.82,0.0712871,-4.367005,0.039604,2.312093,0.0891089,N/A
2,AAP,176.88,N/A,24.64,0.473267,3.20,0.418812,1.15,0.131683,11.649829,0.255446,2.640324,0.108911,N/A
3,AAPL,126.79,N/A,34.20,0.70297,32.20,0.954455,7.24,0.80198,24.378196,0.774257,18.617930,0.922772,N/A
4,ABBV,109.68,N/A,39.32,0.762376,14.73,0.881188,4.31,0.59505,14.145450,0.384158,8.784794,0.534653,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.91,N/A,35.14,0.716832,-4.10,0.0435644,5.56,0.711881,23.927399,0.762376,10.198238,0.673267,N/A
501,ZBH,162.39,N/A,-240.98,0.00990099,2.67,0.349505,4.64,0.615842,29.243242,0.853465,8.119188,0.489109,N/A
502,ZBRA,473.53,N/A,51.50,0.851485,11.81,0.845545,5.73,0.729703,33.214935,0.889109,13.167546,0.784158,N/A
503,ZION,57.51,N/A,9.25,0.180198,1.33,0.136634,2.68,0.39802,7.056707,0.0693069,3.018850,0.126733,N/A


## Calculating the RV Score

In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe    

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.08,N/A,48.07,0.829703,7.60,0.737624,6.70,0.775248,31.154625,0.879208,13.060984,0.782178,0.800792
1,AAL,21.60,N/A,-1.19,0.156436,-2.09,0.0455446,0.82,0.0712871,-4.367005,0.039604,2.312093,0.0891089,0.080396
2,AAP,176.88,N/A,24.64,0.473267,3.20,0.418812,1.15,0.131683,11.649829,0.255446,2.640324,0.108911,0.277624
3,AAPL,126.79,N/A,34.20,0.70297,32.20,0.954455,7.24,0.80198,24.378196,0.774257,18.617930,0.922772,0.831287
4,ABBV,109.68,N/A,39.32,0.762376,14.73,0.881188,4.31,0.59505,14.145450,0.384158,8.784794,0.534653,0.631485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.91,N/A,35.14,0.716832,-4.10,0.0435644,5.56,0.711881,23.927399,0.762376,10.198238,0.673267,0.581584
501,ZBH,162.39,N/A,-240.98,0.00990099,2.67,0.349505,4.64,0.615842,29.243242,0.853465,8.119188,0.489109,0.463564
502,ZBRA,473.53,N/A,51.50,0.851485,11.81,0.845545,5.73,0.729703,33.214935,0.889109,13.167546,0.784158,0.82
503,ZION,57.51,N/A,9.25,0.180198,1.33,0.136634,2.68,0.39802,7.056707,0.0693069,3.018850,0.126733,0.182178


## Selecting the 50 Best Value Stocks¶

In [18]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.60,N/A,7.35,0.168317,0.5478,0.0534653,0.4465,0.0217822,3.534840,0.0475248,0.438491,0.0039604,0.0590099
1,MCK,184.20,N/A,-7.41,0.108911,-61.6800,0.0039604,0.1235,0.00594059,7.097001,0.0732673,2.562989,0.10495,0.0594059
2,FTI,8.72,N/A,-1.22,0.154455,0.9524,0.0712871,0.2991,0.00990099,2.640423,0.0455446,1.673212,0.0534653,0.0669307
3,AIG,48.11,N/A,-6.93,0.118812,0.6170,0.0554455,0.9673,0.0871287,7.327557,0.0811881,0.946837,0.0217822,0.0728713
4,ALL,112.70,N/A,6.53,0.164356,1.2000,0.10198,0.7741,0.0594059,2.471410,0.0435644,0.741078,0.0138614,0.0766337
5,MET,60.75,N/A,10.70,0.194059,0.7114,0.0574257,0.7930,0.0653465,5.315328,0.0534653,0.805667,0.0178218,0.0776238
6,AAL,21.60,N/A,-1.19,0.156436,-2.0900,0.0455446,0.8200,0.0712871,-4.367005,0.039604,2.312093,0.0891089,0.080396
7,ABC,111.05,N/A,-7.11,0.116832,-33.8900,0.0118812,0.1189,0.0039604,8.243649,0.10297,3.828989,0.184158,0.0839604
8,F,12.81,N/A,-39.85,0.049505,1.6000,0.181188,0.3885,0.0178218,6.532815,0.0633663,2.894028,0.120792,0.0865347
9,DXC,29.10,N/A,-2.54,0.140594,1.3300,0.136634,0.4143,0.019802,5.417284,0.0554455,2.396495,0.10099,0.0906931


## Calculating the Number of Shares to Buy

In [19]:
portfolio_input()

Enter the value of your portfolio:25000000


In [20]:
position = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position/rv_dataframe.loc[row, 'Price'])
rv_dataframe  

C:\Users\PRATHAM\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.60,16891,7.35,0.168317,0.5478,0.0534653,0.4465,0.0217822,3.534840,0.0475248,0.438491,0.0039604,0.0590099
1,MCK,184.20,2714,-7.41,0.108911,-61.6800,0.0039604,0.1235,0.00594059,7.097001,0.0732673,2.562989,0.10495,0.0594059
2,FTI,8.72,57339,-1.22,0.154455,0.9524,0.0712871,0.2991,0.00990099,2.640423,0.0455446,1.673212,0.0534653,0.0669307
3,AIG,48.11,10392,-6.93,0.118812,0.6170,0.0554455,0.9673,0.0871287,7.327557,0.0811881,0.946837,0.0217822,0.0728713
4,ALL,112.70,4436,6.53,0.164356,1.2000,0.10198,0.7741,0.0594059,2.471410,0.0435644,0.741078,0.0138614,0.0766337
5,MET,60.75,8230,10.70,0.194059,0.7114,0.0574257,0.7930,0.0653465,5.315328,0.0534653,0.805667,0.0178218,0.0776238
6,AAL,21.60,23148,-1.19,0.156436,-2.0900,0.0455446,0.8200,0.0712871,-4.367005,0.039604,2.312093,0.0891089,0.080396
7,ABC,111.05,4502,-7.11,0.116832,-33.8900,0.0118812,0.1189,0.0039604,8.243649,0.10297,3.828989,0.184158,0.0839604
8,F,12.81,39032,-39.85,0.049505,1.6000,0.181188,0.3885,0.0178218,6.532815,0.0633663,2.894028,0.120792,0.0865347
9,DXC,29.10,17182,-2.54,0.140594,1.3300,0.136634,0.4143,0.019802,5.417284,0.0554455,2.396495,0.10099,0.0906931


## Formatting Our Excel Output

In [21]:
writer = pd.ExcelWriter('Value Strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template], 
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', percent_template],
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

## Saving Our Excel Output

In [24]:
writer.save()